# 사용 패키지

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

In [2]:
# 대회 평가지표
def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

# 데이터 로드

In [3]:
url = 'C:/Users/WIN/Downloads/가스공사/'

In [4]:
total = pd.read_csv(url + '한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')

In [5]:
total.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


# 전처리

In [6]:
total['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [7]:
# 구분을 순자로 변경
d_map = {}
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)

In [8]:
d_map

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'G': 5, 'H': 6}

In [9]:
total

,연월일,시간,구분,공급량
0,2013-01-01,1,0,2497.129
1,2013-01-01,2,0,2363.265
2,2013-01-01,3,0,2258.505
3,2013-01-01,4,0,2243.969
4,2013-01-01,5,0,2344.105
...,...,...,...,...
368083,2018-12-31,20,6,681.033
368084,2018-12-31,21,6,669.961
368085,2018-12-31,22,6,657.941
368086,2018-12-31,23,6,610.953


In [10]:
total['연월일'] = pd.to_datetime(total['연월일'])

In [11]:
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [12]:
total

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0
368084,2018-12-31,21,6,669.961,2018,12,31,0
368085,2018-12-31,22,6,657.941,2018,12,31,0
368086,2018-12-31,23,6,610.953,2018,12,31,0


## 1,2,3월만으로 재구성

In [13]:
mon = total['month'].isin([1,2,3])
total1 = total[mon]
total1

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
361483,2018-03-31,20,6,244.162,2018,3,31,5
361484,2018-03-31,21,6,248.059,2018,3,31,5
361485,2018-03-31,22,6,231.181,2018,3,31,5
361486,2018-03-31,23,6,199.022,2018,3,31,5


## 전처리 마무리

In [14]:
# 2018년만 때어서
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

In [78]:
train_years = [2013,2014,2015,2016,2017,2018]
val_years = [2013,2014,2015,2016,2017,2018]

In [79]:
# 2018년으로 validation set을 생성
train = total1[total1['year'].isin(train_years)]
val = total1[total1['year'].isin(val_years)]

In [80]:
train

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
361483,2018-03-31,20,6,244.162,2018,3,31,5
361484,2018-03-31,21,6,248.059,2018,3,31,5
361485,2018-03-31,22,6,231.181,2018,3,31,5
361486,2018-03-31,23,6,199.022,2018,3,31,5


In [81]:
features = ['구분', 'month', 'day', 'weekday', '시간']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

In [82]:
train_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
361483,6,3,31,5,20
361484,6,3,31,5,21
361485,6,3,31,5,22
361486,6,3,31,5,23


In [83]:
train_y

0         2497.129
1         2363.265
2         2258.505
3         2243.969
4         2344.105
            ...   
361483     244.162
361484     248.059
361485     231.181
361486     199.022
361487     190.212
Name: 공급량, Length: 90888, dtype: float64

# 학습

## 기본 LGBM 모델

In [173]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}

model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=50)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000415 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 75
[LightGBM] [Info] Number of data points in the train set: 75768, number of used features: 5
[LightGBM] [Info] Start training from score 1539.324886
Training until validation scores don't improve for 50 rounds
[20]	valid_0's l1: 343.263
[40]	valid_0's l1: 278.069
[60]	valid_0's l1: 263.963
[80]	valid_0's l1: 259.567
[100]	valid_0's l1: 258.498
[120]	valid_0's l1: 257.487
[140]	valid_0's l1: 259.921
[160]	valid_0's l1: 260.794
Early stopping, best iteration is:
[126]	valid_0's l1: 257.202


C:\Users\WIN\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\WIN\anaconda3\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


In [174]:
pre = model.predict(val_x)
pre

array([1848.41971502, 1660.21747756, 1580.79778636, ...,  422.84451122,
        398.89897323,  364.58076495])

In [175]:
# 평가지표 활용
nmae( pd.DataFrame(val_y).reset_index(drop=True).reset_index() , pd.DataFrame(pre).reset_index())

0.16689029228933744

## KNNR 모델

In [164]:
# 기본
# 1K값을 바꾸어도 결과는 같음 ㅋㅋㅋ
from sklearn.neighbors import KNeighborsRegressor

# 모델 훈련
model = KNeighborsRegressor(n_neighbors=5, weights='distance', p=1, n_jobs=-1) #601
model.fit(train_x, train_y)

KNeighborsRegressor(n_jobs=-1, p=1, weights='distance')

In [165]:
pre = model.predict(val_x)
pre

array([1846.558   , 1782.049   , 1661.4015  , ...,  681.109125,
        347.398   ,  323.415   ])

In [171]:
# 평가지표 활용
nmae( pd.DataFrame(val_y).reset_index(drop=True).reset_index() , pd.DataFrame(pre).reset_index())

0.1810682444276462

## 딥러닝 저장 메소드

In [249]:
# 모델 저장하기
from keras.models import load_model

model.save(url + '/model/DeepAll_176.h5')

In [302]:
# 모델 불러오기
from keras.models import load_model

model = load_model(url + '/model/DeepAll_176.h5')

## 기본 딥러닝

In [84]:
train_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
361483,6,3,31,5,20
361484,6,3,31,5,21
361485,6,3,31,5,22
361486,6,3,31,5,23


In [165]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='linear', input_shape=(5,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))

In [303]:
import tensorflow as tf 

model.compile(optimizer='Adam',
             #optimizer= tf.optimizers.Adamax(lr=0.1, beta_1=0.9, beta_2=0.999) ,
             loss='MAE',
             metrics=['mae', 'mse'])

In [ ]:
history = model.fit(train_x, train_y, epochs=1000, batch_size=128, validation_data=(val_x, val_y))

Epoch 1/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.1861 - mae: 178.1861 - mse: 79800.2734 - val_loss: 177.3718 - val_mae: 177.3718 - val_mse: 79517.9531
Epoch 2/1000
711/711 [==============================] - 1s 982us/step - loss: 178.2879 - mae: 178.2879 - mse: 79831.5703 - val_loss: 177.6488 - val_mae: 177.6488 - val_mse: 79396.2812
Epoch 3/1000
711/711 [==============================] - 1s 972us/step - loss: 178.1840 - mae: 178.1840 - mse: 79833.2344 - val_loss: 177.5743 - val_mae: 177.5743 - val_mse: 79023.5312
Epoch 4/1000
711/711 [==============================] - 1s 990us/step - loss: 178.4170 - mae: 178.4170 - mse: 79743.3281 - val_loss: 177.7081 - val_mae: 177.7081 - val_mse: 79575.4375
Epoch 5/1000
711/711 [==============================] - 1s 994us/step - loss: 178.2243 - mae: 178.2243 - mse: 79765.6562 - val_loss: 177.9909 - val_mae: 177.9909 - val_mse: 79920.0156
Epoch 6/1000
711/711 [==============================] - 1s 970us/step - loss: 178.

711/711 [==============================] - 1s 979us/step - loss: 178.3648 - mae: 178.3648 - mse: 79720.2422 - val_loss: 178.9777 - val_mae: 178.9777 - val_mse: 79515.5547
Epoch 46/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.2087 - mae: 178.2087 - mse: 79776.6406 - val_loss: 177.3487 - val_mae: 177.3487 - val_mse: 79288.6797
Epoch 47/1000
711/711 [==============================] - 1s 976us/step - loss: 178.1032 - mae: 178.1032 - mse: 79714.9844 - val_loss: 178.5785 - val_mae: 178.5785 - val_mse: 79716.5234
Epoch 48/1000
711/711 [==============================] - 1s 997us/step - loss: 178.2130 - mae: 178.2130 - mse: 79763.2188 - val_loss: 180.0488 - val_mae: 180.0488 - val_mse: 80405.1250
Epoch 49/1000
711/711 [==============================] - 1s 956us/step - loss: 178.2047 - mae: 178.2047 - mse: 79676.0078 - val_loss: 180.9519 - val_mae: 180.9519 - val_mse: 80706.5312
Epoch 50/1000
711/711 [==============================] - 1s 996us/step - loss: 178.1143 - m

711/711 [==============================] - 1s 1ms/step - loss: 178.0761 - mae: 178.0761 - mse: 79699.9922 - val_loss: 178.1193 - val_mae: 178.1193 - val_mse: 79840.6797
Epoch 90/1000
711/711 [==============================] - 1s 991us/step - loss: 178.1920 - mae: 178.1920 - mse: 79690.3906 - val_loss: 177.6244 - val_mae: 177.6244 - val_mse: 79028.3750
Epoch 91/1000
711/711 [==============================] - 1s 975us/step - loss: 178.2092 - mae: 178.2092 - mse: 79685.9219 - val_loss: 177.9944 - val_mae: 177.9944 - val_mse: 79732.4219
Epoch 92/1000
711/711 [==============================] - 1s 980us/step - loss: 178.0866 - mae: 178.0866 - mse: 79743.7500 - val_loss: 177.9812 - val_mae: 177.9812 - val_mse: 79253.4375
Epoch 93/1000
711/711 [==============================] - 1s 972us/step - loss: 178.1273 - mae: 178.1273 - mse: 79698.4453 - val_loss: 177.7083 - val_mae: 177.7083 - val_mse: 79003.1484
Epoch 94/1000
711/711 [==============================] - 1s 982us/step - loss: 178.0925 - m

711/711 [==============================] - 1s 992us/step - loss: 178.4081 - mae: 178.4081 - mse: 79737.6484 - val_loss: 177.9279 - val_mae: 177.9279 - val_mse: 79212.2422
Epoch 134/1000
711/711 [==============================] - 1s 976us/step - loss: 178.1358 - mae: 178.1358 - mse: 79633.8438 - val_loss: 177.7944 - val_mae: 177.7944 - val_mse: 79798.6797
Epoch 135/1000
711/711 [==============================] - 1s 976us/step - loss: 178.0839 - mae: 178.0839 - mse: 79660.2422 - val_loss: 177.1847 - val_mae: 177.1847 - val_mse: 79606.8828
Epoch 136/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.1101 - mae: 178.1101 - mse: 79627.3516 - val_loss: 178.1324 - val_mae: 178.1324 - val_mse: 79950.8672
Epoch 137/1000
711/711 [==============================] - 1s 990us/step - loss: 178.0010 - mae: 178.0010 - mse: 79632.1875 - val_loss: 177.5497 - val_mae: 177.5497 - val_mse: 79379.3594
Epoch 138/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.1756 

711/711 [==============================] - 1s 986us/step - loss: 178.0062 - mae: 178.0062 - mse: 79552.3750 - val_loss: 177.6332 - val_mae: 177.6332 - val_mse: 79747.6484
Epoch 178/1000
711/711 [==============================] - 1s 999us/step - loss: 177.9226 - mae: 177.9226 - mse: 79579.1797 - val_loss: 177.1469 - val_mae: 177.1469 - val_mse: 78995.8594
Epoch 179/1000
711/711 [==============================] - 1s 1ms/step - loss: 177.9644 - mae: 177.9644 - mse: 79572.3281 - val_loss: 177.3125 - val_mae: 177.3125 - val_mse: 79309.0391
Epoch 180/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.0353 - mae: 178.0353 - mse: 79514.7031 - val_loss: 177.5994 - val_mae: 177.5994 - val_mse: 79468.9219
Epoch 181/1000
711/711 [==============================] - 1s 983us/step - loss: 178.0166 - mae: 178.0166 - mse: 79583.0391 - val_loss: 177.4586 - val_mae: 177.4586 - val_mse: 79552.6797
Epoch 182/1000
711/711 [==============================] - 1s 979us/step - loss: 178.0415 

...

711/711 [==============================] - 1s 1ms/step - loss: 177.9498 - mae: 177.9498 - mse: 79619.2109 - val_loss: 179.3830 - val_mae: 179.3830 - val_mse: 80223.4219
Epoch 266/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.0293 - mae: 178.0293 - mse: 79548.8359 - val_loss: 177.2038 - val_mae: 177.2038 - val_mse: 79146.7500
Epoch 267/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.0492 - mae: 178.0492 - mse: 79607.6719 - val_loss: 177.4136 - val_mae: 177.4136 - val_mse: 79247.3203
Epoch 268/1000
711/711 [==============================] - 1s 979us/step - loss: 178.0676 - mae: 178.0676 - mse: 79613.2500 - val_loss: 179.6867 - val_mae: 179.6867 - val_mse: 79647.9688
Epoch 269/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.1403 - mae: 178.1403 - mse: 79514.5859 - val_loss: 178.7040 - val_mae: 178.7040 - val_mse: 79602.2344
Epoch 270/1000
711/711 [==============================] - 1s 1ms/step - loss: 177.8424 - mae:

711/711 [==============================] - 1s 1ms/step - loss: 177.8503 - mae: 177.8503 - mse: 79463.7188 - val_loss: 177.6539 - val_mae: 177.6539 - val_mse: 79553.0469
Epoch 310/1000
711/711 [==============================] - 1s 1ms/step - loss: 177.9501 - mae: 177.9501 - mse: 79579.8438 - val_loss: 177.8611 - val_mae: 177.8611 - val_mse: 79253.9922
Epoch 311/1000
711/711 [==============================] - 1s 972us/step - loss: 178.1100 - mae: 178.1100 - mse: 79642.1172 - val_loss: 177.3162 - val_mae: 177.3162 - val_mse: 79395.5781
Epoch 312/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.0333 - mae: 178.0333 - mse: 79601.9844 - val_loss: 177.6182 - val_mae: 177.6182 - val_mse: 79458.1875
Epoch 313/1000
711/711 [==============================] - 1s 1ms/step - loss: 177.9436 - mae: 177.9436 - mse: 79590.4609 - val_loss: 177.2564 - val_mae: 177.2564 - val_mse: 79608.4609
Epoch 314/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.0411 - mae:

711/711 [==============================] - 1s 958us/step - loss: 178.3463 - mae: 178.3463 - mse: 79579.3359 - val_loss: 177.6460 - val_mae: 177.6460 - val_mse: 79331.4922
Epoch 354/1000
711/711 [==============================] - 1s 975us/step - loss: 177.9231 - mae: 177.9231 - mse: 79538.4297 - val_loss: 178.1787 - val_mae: 178.1787 - val_mse: 79909.6328
Epoch 355/1000
711/711 [==============================] - 1s 984us/step - loss: 178.0544 - mae: 178.0544 - mse: 79615.4141 - val_loss: 177.5307 - val_mae: 177.5307 - val_mse: 79491.4141
Epoch 356/1000
711/711 [==============================] - 1s 969us/step - loss: 178.0317 - mae: 178.0317 - mse: 79504.4219 - val_loss: 177.5666 - val_mae: 177.5666 - val_mse: 79732.2109
Epoch 357/1000
711/711 [==============================] - 1s 970us/step - loss: 177.9265 - mae: 177.9265 - mse: 79518.5781 - val_loss: 179.6667 - val_mae: 179.6667 - val_mse: 79474.0156
Epoch 358/1000
711/711 [==============================] - 1s 972us/step - loss: 178.1

711/711 [==============================] - 1s 993us/step - loss: 177.9415 - mae: 177.9415 - mse: 79503.3594 - val_loss: 177.6228 - val_mae: 177.6228 - val_mse: 79298.7578
Epoch 398/1000
711/711 [==============================] - 1s 965us/step - loss: 177.8600 - mae: 177.8600 - mse: 79617.4844 - val_loss: 181.3093 - val_mae: 181.3093 - val_mse: 80563.5703
Epoch 399/1000
711/711 [==============================] - 1s 1ms/step - loss: 178.0973 - mae: 178.0973 - mse: 79569.9375 - val_loss: 177.0874 - val_mae: 177.0874 - val_mse: 79217.3516
Epoch 400/1000
711/711 [==============================] - 1s 997us/step - loss: 177.9450 - mae: 177.9450 - mse: 79595.7422 - val_loss: 179.5913 - val_mae: 179.5913 - val_mse: 80170.3828
Epoch 401/1000
711/711 [==============================] - 1s 972us/step - loss: 178.0599 - mae: 178.0599 - mse: 79608.9609 - val_loss: 177.3946 - val_mae: 177.3946 - val_mse: 79486.9375
Epoch 402/1000
711/711 [==============================] - 1s 990us/step - loss: 178.096

711/711 [==============================] - 1s 983us/step - loss: 177.9018 - mae: 177.9018 - mse: 79515.0547 - val_loss: 179.3546 - val_mae: 179.3546 - val_mse: 79903.9219
Epoch 442/1000
711/711 [==============================] - 1s 968us/step - loss: 177.8645 - mae: 177.8645 - mse: 79541.2266 - val_loss: 177.6939 - val_mae: 177.6939 - val_mse: 79402.3125
Epoch 443/1000
711/711 [==============================] - 1s 990us/step - loss: 178.1393 - mae: 178.1393 - mse: 79554.1484 - val_loss: 176.8940 - val_mae: 176.8940 - val_mse: 79242.5156
Epoch 444/1000
711/711 [==============================] - 1s 990us/step - loss: 178.0028 - mae: 178.0028 - mse: 79514.4297 - val_loss: 177.3073 - val_mae: 177.3073 - val_mse: 79777.2734
Epoch 445/1000
711/711 [==============================] - 1s 965us/step - loss: 177.8818 - mae: 177.8818 - mse: 79459.1562 - val_loss: 179.7541 - val_mae: 179.7541 - val_mse: 79645.1250
Epoch 446/1000
711/711 [==============================] - 1s 979us/step - loss: 177.8

In [ ]:
# 모델 loss 그래프

fit_loss = history.history['mae']
# fit_loss2 = history.history['mse']

x_len = np.arange(len(fit_loss))
plt.plot(x_len, fit_loss, marker=',', c='red', label='MAE')
# plt.plot(x_len, fit_loss2, marker=',', c='blue', label='MSE')

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
pre = model.predict(val_x)
pre

In [ ]:
nmae(pd.DataFrame(val_y).reset_index(drop=True).reset_index(),pd.DataFrame(pre).reset_index())

In [148]:
# d_train = lgb.Dataset(train_x, train_y)
# d_val = lgb.Dataset(val_x, val_y)

# params = {
#     'objective': 'regression',
#     'metric':'mae',
#     'seed':42
# }

# model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)

C:\Users\WIN\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\WIN\anaconda3\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000737 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 75
[LightGBM] [Info] Number of data points in the train set: 90888, number of used features: 5
[LightGBM] [Info] Start training from score 1565.788903
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 257.626
[40]	valid_0's l1: 190.049
[60]	valid_0's l1: 170.322
[80]	valid_0's l1: 160.565
[100]	valid_0's l1: 154.2
[120]	valid_0's l1: 149.138
[140]	valid_0's l1: 145.172
[160]	valid_0's l1: 140.636
[180]	valid_0's l1: 136.419
[200]	valid_0's l1: 132.174
[220]	valid_0's l1: 127.898
[240]	valid_0's l1: 123.783
[260]	valid_0's l1: 120.119
[280]	valid_0's l1: 116.587
[300]	valid_0's l1: 113.59
[320]	valid_0's l1: 111.066
[340]	valid_0's l1: 109.122
[360]	valid_0's l1: 107.191
[380]	valid_0's l1: 104.657
[400]

# 추론 및 결과 제출

In [97]:
test = pd.read_csv(url + 'test.csv')
submission = pd.read_csv(url + 'sample_submission.csv')

In [98]:
test.head()

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A


In [99]:
submission.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [100]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [101]:
test

,일자|시간|구분,일자,시간,구분
0,2019-01-01 01 A,2019-01-01,1,A
1,2019-01-01 02 A,2019-01-01,2,A
2,2019-01-01 03 A,2019-01-01,3,A
3,2019-01-01 04 A,2019-01-01,4,A
4,2019-01-01 05 A,2019-01-01,5,A
...,...,...,...,...
15115,2019-03-31 20 H,2019-03-31,20,H
15116,2019-03-31 21 H,2019-03-31,21,H
15117,2019-03-31 22 H,2019-03-31,22,H
15118,2019-03-31 23 H,2019-03-31,23,H


In [102]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [103]:
test

,일자|시간|구분,일자,시간,구분,year,month,day,weekday
0,2019-01-01 01 A,2019-01-01,1,A,2019,1,1,1
1,2019-01-01 02 A,2019-01-01,2,A,2019,1,1,1
2,2019-01-01 03 A,2019-01-01,3,A,2019,1,1,1
3,2019-01-01 04 A,2019-01-01,4,A,2019,1,1,1
4,2019-01-01 05 A,2019-01-01,5,A,2019,1,1,1
...,...,...,...,...,...,...,...,...
15115,2019-03-31 20 H,2019-03-31,20,H,2019,3,31,6
15116,2019-03-31 21 H,2019-03-31,21,H,2019,3,31,6
15117,2019-03-31 22 H,2019-03-31,22,H,2019,3,31,6
15118,2019-03-31 23 H,2019-03-31,23,H,2019,3,31,6


In [104]:
test['구분'].map

<bound method Series.map of 0        A
1        A
2        A
3        A
4        A
        ..
15115    H
15116    H
15117    H
15118    H
15119    H
Name: 구분, Length: 15120, dtype: object>

In [105]:
test['구분'] = test['구분'].map(d_map)

In [106]:
test_x = test[features]

In [107]:
test_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
15115,6,3,31,6,20
15116,6,3,31,6,21
15117,6,3,31,6,22
15118,6,3,31,6,23


In [108]:
# preds = model.predict(test_x)
preds = model.predict(test_x)

In [109]:
preds

array([[2051.6062 ],
       [2137.2012 ],
       [2222.7964 ],
       ...,
       [ 284.3889 ],
       [ 284.49387],
       [ 284.59933]], dtype=float32)

In [110]:
submission['공급량'] = preds

In [111]:
submission.to_csv(url + '/output/DeepA_279.csv', index=False)